<a href="https://colab.research.google.com/github/arishp/veltech_genai/blob/main/BERT_extractive_question_answering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from transformers import(
    BertForQuestionAnswering,
    BertTokenizerFast,
)
from scipy.special import softmax
import plotly.express as px
import pandas as pd
import numpy as np

In [ ]:
context = "The giraffe is a large African hoofed mammal belonging to the genus Giraffa. It is the tallest living terrestrial animal and the largest ruminant on Earth. Traditionally, giraffes have been thought of as one species, Giraffa camelopardalis, with nine subspecies. Most recently, researchers proposed dividing them into up to eight extant species due to new research into their mitochondrial and nuclear DNA, as well as morphological measurements. Seven other extinct species of Giraffa are known from the fossil record."

In [ ]:
question = "How many giraffe species are there?"

In [ ]:
model_name = "deepset/bert-base-cased-squad2"
tokenizer = BertTokenizerFast.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/152 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at deepset/bert-base-cased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
inputs = tokenizer(question, context, return_tensors="pt")
with torch.no_grad():
  outputs = model(**inputs)

In [ ]:
start_scores, end_scores = softmax(outputs.start_logits)[0], softmax(outputs.end_logits)[0]

In [ ]:
start_scores

array([7.70189688e-02, 8.29706551e-05, 4.57422539e-05, 1.11435205e-04,
       3.98007578e-05, 2.93639241e-05, 2.45868050e-05, 4.72943320e-05,
       5.39923094e-05, 5.99460727e-05, 2.91368033e-05, 5.48411481e-05,
       2.47972109e-03, 5.34964202e-05, 2.86079710e-04, 4.56485104e-05,
       2.35064697e-04, 7.00822857e-04, 1.24515122e-04, 2.58674554e-04,
       4.78287438e-05, 2.29203542e-05, 1.95467845e-04, 1.08255845e-04,
       5.59246109e-05, 3.27736707e-05, 4.56542621e-05, 6.49553986e-05,
       1.92232343e-04, 3.80510974e-05, 7.85467273e-05, 2.57776646e-05,
       4.89469021e-05, 3.52646312e-04, 5.80711967e-05, 1.09928216e-04,
       5.33404585e-04, 6.41510924e-05, 1.41073790e-04, 3.43564388e-05,
       3.18624807e-05, 7.27558217e-05, 1.68711907e-04, 1.08407708e-04,
       3.77810575e-05, 1.55917933e-05, 2.78484931e-05, 8.27102776e-05,
       3.85608473e-05, 1.09772875e-04, 6.82466780e-05, 1.34716379e-02,
       8.81959786e-05, 1.54121051e-04, 4.87722609e-05, 9.15999481e-05,
      

In [ ]:
start_idx = np.argmax(start_scores)
end_idx = np.argmax(end_scores)
print(start_idx)
print(end_idx)

86
86


In [ ]:
answer_ids = inputs.input_ids[0][start_idx: end_idx+1]
answer_tokens = tokenizer.convert_ids_to_tokens(answer_ids)
answer = tokenizer.convert_tokens_to_string(answer_tokens)
print(answer)

eight


In [ ]:
def predict_answer(context, question):
  inputs = tokenizer(question, context, return_tensors="pt")
  with torch.no_grad():
    outputs = model(**inputs)
  start_scores, end_scores = softmax(outputs.start_logits)[0], softmax(outputs.end_logits)[0]
  start_idx = np.argmax(start_scores)
  end_idx = np.argmax(end_scores)
  confidence_score = (start_scores[start_idx] + end_scores[end_idx])/2
  answer_ids = inputs.input_ids[0][start_idx: end_idx+1]
  answer_tokens = tokenizer.convert_ids_to_tokens(answer_ids)
  answer = tokenizer.convert_tokens_to_string(answer_tokens)
  if answer != tokenizer.cls_token:
    return answer, confidence_score
  return None, confidence_score

In [ ]:
context = "Coffee is a beverage brewed from roasted coffee beans. Darkly colored, bitter, and slightly acidic, coffee has a stimulating effect on humans, primarily due to its caffeine content. It has the highest sales in the world market for hot drinks."

In [ ]:
question = "what is coffee?"

In [ ]:
predict_answer(context, question)

('a beverage brewed from roasted coffee beans', 0.8783529996871948)

In [ ]:
question = "what are the most common coffee beans?"
predict_answer(context, question)

(None, 0.9983177185058594)

In [ ]:
question = "how can i make iced coffee?"
predict_answer(context, question)

(None, 0.8096498250961304)